Unpause the last notebook

# 6 Composition with ANCOM

## 6.1 ANCOM by ASV

Beta-diversities are OK to plot our samples in space, but if we want to really see if two groups are really different we need to do more statistical tests.

To look for these statistical differences we are going to use ANCOM

For example, we want to know:
- which ASVs are the most different in *gut* between the two *subjects*

This procedure works in multiple steps:
1. Filter the OTU table so you only have the **gut** samples
2. Apply a small correction to the values of the OTU table
3. Run ANCOM over the two **subjects**

Let's filter the OTU table so there are only gut samples

In [3]:
qiime feature-table filter-samples \
    --i-table          table.qza \
    --m-metadata-file  sample-metadata.tsv \
    --p-where          "[body-site]='gut'" \
    --o-filtered-table gut-table.qza

Saved FeatureTable[Frequency] to: gut-table.qza



Then, fix the table with pseudo-counts, becasue ANCOM cannot work with zeros in the table.

In [6]:
qiime composition add-pseudocount \
    --i-table             gut-table.qza \
    --o-composition-table comp-gut-table.qza

Saved FeatureTable[Composition] to: comp-gut-table.qza



Finally, run ANCOM, specifying that we want the analyses between each `subject`

In [7]:
qiime composition ancom \
    --i-table           comp-gut-table.qza \
    --m-metadata-file   sample-metadata.tsv \
    --m-metadata-column subject \
    --o-visualization   ancom-subject.qzv

Saved Visualization to: ancom-subject.qzv



- [ancom-subject.qzv](https://view.qiime2.org/?src=https%3A%2F%2Fdocs.qiime2.org%2F2023.2%2Fdata%2Ftutorials%2Fmoving-pictures%2Fancom-subject.qzv)

### Exercise

- Which ASVs are the most different between the two subjects in the gut?

- What are the taxonomies of these ASVs? (Pssst: the rep-seqs table is [here](https://view.qiime2.org/visualization/?type=html&src=https%3A%2F%2Fdocs.qiime2.org%2F2023.2%2Fdata%2Ftutorials%2Fmoving-pictures%2Frep-seqs.qzv))

## 6.2 ANCOM by taxa

Instead of finding the most characteristic ASV of each group, we are really interesed in which taxonomic group (kingdom, order, phylum) is most different between samples.

In this case, the procedure is simmilar, with an extra step:
1. Filter the body site (already done: gut-table.qza)
2. Collapse by taxon level
3. Apply correction
4. Run ANCOM

In [ ]:
# This is already done
# qiime feature-table filter-samples \
#    --i-table          table.qza \
#    --m-metadata-file  sample-metadata.tsv \
#    --p-where          "[body-site]='gut'" \
#    --o-filtered-table gut-table.qza

In this case, we will need the taxonomy.qza we generated in the previous notebook.

We are collapsing ASVs to the 6th level (genus).

In [14]:
qiime taxa collapse \
  --i-table           gut-table.qza \
  --i-taxonomy        taxonomy.qza \
  --p-level           6 \
  --o-collapsed-table gut-table-l6.qza

Saved FeatureTable[Frequency] to: gut-table-l6.qza



Add pseudocounts

In [15]:
qiime composition add-pseudocount \
  --i-table             gut-table-l6.qza \
  --o-composition-table comp-gut-table-l6.qza

Saved FeatureTable[Composition] to: comp-gut-table-l6.qza



In [ ]:
And finally, run ANCOM:

In [16]:
qiime composition ancom \
  --i-table           comp-gut-table-l6.qza \
  --m-metadata-file   sample-metadata.tsv \
  --m-metadata-column subject \
  --o-visualization   l6-ancom-subject.qzv

Saved Visualization to: l6-ancom-subject.qzv



- [l6-ancom-subject.qzv](https://view.qiime2.org/?src=https%3A%2F%2Fdocs.qiime2.org%2F2023.2%2Fdata%2Ftutorials%2Fmoving-pictures%2Fl6-ancom-subject.qzv)

### Exercise

- What genera are differentially abundant between the two samples?